In [1]:
import pandas as pd
import numpy as np
import requests, json 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from time import sleep

### Airports

In [225]:
airports = pd.read_csv('/Users/macbook/Downloads/Immobel/airport_df.csv')
airports.shape

(7698, 14)

In [6]:
airports['Country'].nunique(), airports['City'].nunique()

(237, 6955)

In [226]:
wrong_ports = ['Base','Municipal','Field','Airfield','Heliport', 'Airstrip',
               'Force','Army', 'Naval','Military','RAF', 'Aerodrome']

In [227]:
airports['Name'] = airports['Name'].apply(lambda x: np.nan if any([w.lower() in x.lower() for w in wrong_ports]) else x)
airports['IATA'] = airports['IATA'].apply(lambda x: np.nan if x == '\\N' else x)

In [228]:
airports = airports.dropna(subset=['Name','IATA'])
airports.shape

(5397, 14)

### Addresses

In [275]:
addresses = pd.read_csv('immobel_addresses.csv')

In [373]:
addresses.head()

,id,street,city,state,postcode,lat,lon
0,1.98134e+07,2160 Wood Hollow Ct,San Jose,CA,95138.0,37.2862,-121.774
1,1.86869e+07,9516 N Senator Dr,Fresno,CA,93720.0,36.8745,-119.765
2,1.96959e+07,4541 Abbeygate Ct,San Jose,CA,95124.0,37.2554,-121.906
3,2.58512e+07,5845 Stallon Way,Sacramento,CA,95823.0,38.4825,-121.435
4,2.4811e+07,6209 Rockwell St,Oakland,CA,94618.0,37.8506,-122.249


In [374]:
addresses.shape

(200, 7)

### Calling API and structuring API responses

In [377]:
addresses['Metadata'] = np.nan
addresses['Origin Address'] = np.nan
api_key = 'API KEY'

In [ ]:
%%time
for n in addresses.index[:]:   
    
    # Preparing data for request
    dest = ''
    org_lat = addresses['lat'].loc[n]
    org_lon = addresses['lon'].loc[n]
    
    airport_filter = airports[(airports['Lat'] < org_lat+0.9) & 
                                (airports['Lat'] > org_lat-0.9) & 
                                (airports['Long'] < org_lon+1.2) & 
                                (airports['Long'] > org_lon-1.2)]

    for idx in airport_filter.index:
        coord = str(airports['Lat'].loc[idx]) + '%2C' + str(airports['Long'].loc[idx])
        dest = dest + coord + '%7C'
    
    # API Request
    origin = str(org_lat) + ',' + str(org_lon)
    url = 'https://maps.googleapis.com/maps/api/distancematrix/json?&origins={}&destinations={}&key={}'.format(origin, dest, api_key)
    r = requests.get(url)
    
    
    # Processing request
    try:
        addresses['Origin Address'].loc[n] = r.json()['origin_addresses'][0]
    except:
        pass
    
    air_list = []
    for i in range(airport_filter.shape[0]):

        air_dict = {}
        
        try:
            air_dict['Distance, m'] = r.json()['rows'][0]['elements'][i]['distance']['value']
            air_dict['Duration, sec'] = r.json()['rows'][0]['elements'][i]['duration']['value']
        
        except:
            air_dict['Distance, m'] = np.nan
            air_dict['Duration, sec'] = np.nan
            
        air_dict['Destination Address'] = r.json()['destination_addresses'][i]    
        air_dict['Airport Name'] = airport_filter['Name'].values[i]
        air_dict['Airport Code'] = airport_filter['IATA'].values[i]
        air_dict['Airport City'] = airport_filter['City'].values[i]
        
        air_dict['Airport Lat'] = airport_filter['Lat'].values[i]
        air_dict['Airport Lon'] = airport_filter['Long'].values[i]
        air_dict['Origin ID'] = addresses['id'].loc[n]

        air_list.append(air_dict)
    
    air_list = sorted(air_list, key=lambda k: k['Distance, m']) 
    
    addresses['Metadata'].loc[n] = air_list

In [387]:
addresses.head(1)

,id,street,city,state,postcode,lat,lon,Metadata
0,1.98134e+07,2160 Wood Hollow Ct,San Jose,CA,95138.0,37.2862,-121.774,"[{'Distance, m': 9470, 'Duration, sec': 753, '..."


### Destination data

In [383]:
destination_df = pd.DataFrame()
for i in addresses.index:
    meat_df = pd.DataFrame(addresses['Metadata'].loc[i])
    destination_df = pd.concat([destination_df, meat_df], ignore_index=True)

destination_df.shape

(1254, 9)

In [386]:
destination_df.head(1)

,"Distance, m","Duration, sec",Destination Address,Airport Name,Airport Code,Airport City,Airport Lat,Airport Lon,Origin ID
0,9470.0,753.0,"2204 Waverly Ave, San Jose, CA 95122, USA",Reid-Hillview Airport of Santa Clara County,RHV,San Jose,37.332901,-121.819,1.98134e+07


### Filtering Data and choosing listings

In [394]:
# Choose max distance in km to the nearest airports
max_distanace = 10
destination_df[destination_df['Distance, m'] < max_distanace*1000]['Origin ID'].nunique()

32

In [327]:
# IDs of origins 
org_ids = destination_df[destination_df['Distance, m'] < max_distanace*1000]['Origin ID'].unique()

In [ ]:
# How many listings are filtered
addresses[addresses['zpid'].isin(org_ids)].shape

In [329]:
# Filtered listings
addresses[addresses['zpid'].isin(org_ids)]

,zpid,streetTitle,city,state,zipCode,lat,lon,Metadata
7,89475173,809 Lippert Pl,Santa Clara,CA,95050.0,37.3414,-121.968,"[{'Distance, m': 4886, 'Duration, sec': 638, '..."
28,25806689,2341 Hooke Way,Sacramento,CA,95822.0,38.5269,-121.485,"[{'Distance, m': 2944, 'Duration, sec': 314, '..."
41,18781199,5489 E Holland Ave,Fresno,CA,93727.0,36.7980,-119.705,"[{'Distance, m': 3331, 'Duration, sec': 328, '..."
47,25842240,13 Canyon Tree Ct,Sacramento,CA,95822.0,38.4998,-121.474,"[{'Distance, m': 2909, 'Duration, sec': 333, '..."
57,19790700,1866 Indian Creek Ct,San Jose,CA,95148.0,37.3505,-121.795,"[{'Distance, m': 4826, 'Duration, sec': 586, '..."
86,25805760,2370 19th Ave,Sacramento,CA,95822.0,38.5347,-121.484,"[{'Distance, m': 3772, 'Duration, sec': 381, '..."
88,19802275,1783 Tustin Dr,San Jose,CA,95122.0,37.3133,-121.814,"[{'Distance, m': 3124, 'Duration, sec': 392, '..."
98,19468608,1082 Stanley Way,Palo Alto,CA,94303.0,37.4473,-122.137,"[{'Distance, m': 3459, 'Duration, sec': 473, '..."
